# Import

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from pandas import read_csv


import sys
sys.path.append("C:\\Users\zachc\Documents\GitHub")

from hnc.hnc.PIE import Plasma_of_Ions_and_Electrons as PLASMA

from hnc.hnc.constants import *
from hnc.hnc.misc import n_from_rs, rs_from_n

from pandas import read_csv

prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']
                
from IPython.core.display import HTML
HTML("""
<style>
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}

</style>
""")
import matplotlib as mpl
from cycler import cycler
mpl.rcParams['lines.linewidth'] = 2.2
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.top'] = False
mpl.rcParams['axes.labelsize'] = 20
mpl.rcParams['lines.markersize'] = 8
mpl.rcParams['font.sans-serif'] = ['Arial', 'Tahoma', 'DejaVu Sans', 'Lucida Grande', 'Verdana']
mpl.rcParams['font.size'] = 16
mpl.rcParams['axes.grid'] = True
mpl.rcParams['grid.color'] = '#636363'
mpl.rcParams['grid.alpha'] = 0.12
mpl.rcParams['grid.linewidth'] = 0.7
mpl.rcParams['grid.linestyle'] = '-'
mpl.rcParams['axes.prop_cycle'] = cycler('color',  ['#0AB34F', '#bf009f', '#00abc3', '#8e92c5', '#cc7014', '#e32851'])
mpl.rcParams['legend.fancybox'] = True
mpl.rcParams['legend.framealpha'] = 0.6
mpl.rcParams['legend.fontsize'] = 14
mpl.rcParams['figure.figsize'] = 8, 4.9443
mpl.rcParams['figure.dpi'] = 100

colors = ['#0AB34F', '#bf009f', '#00abc3', '#8e92c5', '#cc7014', '#e32851']


ModuleNotFoundError: No module named 'atomic_forces'

# Function that sets up HNC QSP 

### Create and run plasma

In [ ]:
name='H'
Z, A = 1,1
mi = m_p * A

ni_cc = 5.98e23
ri_cm = rs_from_n(ni_cc)
ri_AU = ri_cm*cm_to_AU


### Make βu_ii, βu_fi from using bound density from AA
If we assume the bound state does not deform at all, then we can calculate potentials based on it easily via integration

### Solving

In [ ]:
from scipy.optimize import least_squares
from scipy.interpolate import interp1d,CubicSpline, PchipInterpolator

def custom_flat(hnc_matrix):
    hnc_flat = np.concatenate([hnc_matrix[0,0],hnc_matrix[0,1],hnc_matrix[1,1]])
    return hnc_flat

def custom_unflat(hnc_flat):
    hnc00, hnc01, hnc11 = np.split(hnc_flat,3)
    hnc_matrix = np.zeros((2,2,len(hnc00)))
    hnc_matrix[0,0] = hnc00
    hnc_matrix[1,0] = hnc01
    hnc_matrix[0,1] = hnc01
    hnc_matrix[1,1] = hnc11
    return hnc_matrix

def solve_least_squares(plasma, sparse_k_array, c_s_k_guess=None, ftol=1e-8):
    if c_s_k_guess is None:
        c_s_k_guess = plasma.hnc.c_s_k_matrix

    # Create functions for converting between sparse, not sparse and flattened versions of c_s_k
    get_sparse_csk = lambda c_s_k_flat: np.concatenate([PchipInterpolator(plasma.hnc.k_array, c_s_k)(sparse_k_array) for c_s_k in np.split(c_s_k_flat,3) ])
    get_csk = lambda sparse_hnc_csk: PchipInterpolator(sparse_k_array, sparse_hnc_csk)(plasma.hnc.k_array)
    get_csk_matrix_from_sparse_csk_flat =  lambda sparse_c_s_k_flat: custom_unflat(np.concatenate([get_csk(sparse_csk) for sparse_csk in np.split(sparse_c_s_k_flat,3)]))
    
    def min_function(sparse_c_s_k_flat):
        c_s_k_matrix = get_csk_matrix_from_sparse_csk_flat(sparse_c_s_k_flat)
        new_c_s_k_matrix = plasma.hnc.guess_c_s_k_matrix(c_s_k_matrix)
        plasma.hnc.c_s_k_matrix = new_c_s_k_matrix.copy()
        new_c_s_k_flat = custom_flat(new_c_s_k_matrix)
        return sparse_c_s_k_flat - get_sparse_csk(new_c_s_k_flat)
    
    c_s_k_flat_guess = custom_flat(c_s_k_guess)
    c_s_k_sparse_guess = get_sparse_csk(c_s_k_flat_guess)
    sol = least_squares(min_function, c_s_k_sparse_guess, verbose=2, ftol=ftol)#, bounds = c_s_k_bounds)
    plasma.hnc.c_s_k_matrix = get_csk_matrix_from_sparse_csk_flat(sol.x)
    plasma.hnc.set_all_matrices_from_csk(plasma.hnc.c_s_k_matrix)
    print(sol)

def best_shift(plasma, c_s_k_guess = None, ftol=1e-8, set=True):
    if c_s_k_guess is None:
        c_s_k_guess = plasma.hnc.c_s_k_matrix

    def min_function(multiplier_list):
        xii, xei, xee = multiplier_list
        multiplier_matrix = np.array([[xii,xei],[xei,xee]])
        c_s_k_matrix = multiplier_matrix[:,:,np.newaxis] * c_s_k_guess
        new_c_s_k_matrix = plasma.hnc.guess_c_s_k_matrix(c_s_k_matrix)
        
        return c_s_k_matrix.flatten() - new_c_s_k_matrix.flatten()
    
    sol = least_squares(min_function, np.ones((3)) , verbose=2, ftol=ftol)#, bounds = c_s_k_bounds)
    xii, xei, xee = sol.x
    multiplier_matrix = np.array([[xii,xei],[xei,xee]])
    
    plasma.hnc.c_s_k_matrix = multiplier_matrix[:,:,np.newaxis] * c_s_k_guess
    print(sol)
    return sol

In [ ]:
def plasma_f(T_eV, Rc, Zbar, closure='svt'):
    plasma = PLASMA(Z, A, ni_cc, T_eV, T_eV, Zbar=Zbar, find_βuee=False,
             βu_options = {'add_bridge':True,'bridge':'yukawa', 'pseudopotential':False}, 
             qsp_options = {'Te_c_type':'DMC'},
             hnc_options={'kappa_multiscale':10, 'R_max':10,'N_bins':1000,'oz_method':closure},
             hnc_solve_options={'num_iterations':10000,'alpha_Picard':0.1, 'verbose':False,'iters_to_wait':500},
             root_options={'options':{'maxfev':100,'xtol':1e-14}})

    return plasma

In [ ]:
def ocp_f(T_eV, Zbar):
    ocp = PLASMA(Z, A, ni_cc, T_eV, T_eV, Zbar=Zbar, find_βuee=False,
             βu_options = {'add_bridge':True,'bridge':'yukawa', 'pseudopotential':False}, 
                 hnc_options={'kappa_multiscale':10, 'R_max':10,'N_bins':1000,'oz_method':'standard'},
                 hnc_solve_options={'num_iterations':10000,'alpha_Picard':0.1, 'verbose':False,'iters_to_wait':500},
                 root_options={'options':{'maxfev':100,'xtol':1e-14}})
    ocp.run_ocp_hnc()
    return ocp

def yp_f(T_eV, Zbar):
    yukawa_plasma =  PLASMA(Z, A, ni_cc, T_eV, T_eV, Zbar=Zbar, find_βuee=False,
                 βu_options = {'add_bridge':True,'bridge':'yukawa', 'pseudopotential':False}, 
                 hnc_options={'kappa_multiscale':10, 'R_max':10,'N_bins':1000,'oz_method':'standard'},
                 hnc_solve_options={'num_iterations':10000,'alpha_Picard':0.1, 'verbose':False,'iters_to_wait':500},
                 root_options={'options':{'maxfev':100,'xtol':1e-14}})
    yukawa_plasma.run_yukawa_hnc();
    return yukawa_plasma

In [ ]:
def save_plasma(plasma, Rc):
    T_eV = plasma.Ti_eV
    Zbar = plasma.qsp.Zbar
    ocp = ocp_f( T_eV, Zbar)
    yp  = yp_f(T_eV, Zbar )
    
    # Make header
    info_header = (f"# Zach Johnson Dec/11/2023 CHNC for TCCW Paper" + '\n' + 
                   f"# Ti={T_eV:0.3e}[eV], Tie={plasma.qsp.Tie_c*AU_to_eV:0.3e}[eV] , Te_c={plasma.qsp.Te_c*AU_to_eV:0.3e}[eV], n_i={plasma.ni_cc:0.3e}[1/cc], n_e={Zbar*plasma.ni_cc:0.3e}[1/cc]" + '\n' + 
                   f"# ri={ri_AU:0.3f}, Zbar={Zbar:0.3f}, Rc={Rc:0.3f}[AU], Γii={plasma.qsp.Γii:0.3e}, Γei={plasma.qsp.Γei:0.3e} , Γee={plasma.qsp.Γee:0.3e}, κ={plasma.qsp.get_κ():0.3e}" 
                  )
    column_header = f"{'r/ri':10}  {'g_ii':10}  {'g_ei':10}  {'g_ee':10}  {'OCP:g_ii':10}  {'Yuk:g_ii':10}  {'βu_ii_eff':10}  {'βu_OCP':10}  {'βu_Yuk':10}"
    header = info_header + '\n' + column_header
    
    # Make save array
    save_array = np.array([  plasma.hnc.r_array, plasma.hnc.h_r_matrix[0,0]+1, plasma.hnc.h_r_matrix[0,1]+1, plasma.hnc.h_r_matrix[1,1]+1,
                           ocp.ocp_hnc.h_r_matrix[0,0]+1, yp.yuk_hnc.h_r_matrix[0,0] + 1, plasma.βueff_r_matrix_with_B[0,0],
                           ocp.ocp_hnc.βu_r_matrix[0,0], yp.yuk_hnc.βu_r_matrix[0,0]]).T
    
    np.savetxt(f"/home/zach/plasma/hnc/TCCW/Final_TCCW/results/CHNC_H_rs{ri_AU:0.3f}_TeV{plasma.Ti_eV:0.3f}.dat", save_array, 
               header=header, fmt='%10.5e', comments='')

### Now Load Data from George
Rc is interpreted as the average bound radius, in AU in the file. Empty-core Ashcroft assumed.

In [ ]:
GP_data   = read_csv("/home/zach/plasma/hnc/TCCW/Final_TCCW/GP_data/H_combined.dat", header=0, delim_whitespace=True)
T_eV_list = GP_data['Te(eV)']
Rc_list   = GP_data['Rc'] # AU
Zbar_list = GP_data['Z_bar'] #Average atom Zbar GP_data
GP_data

In [ ]:
plasma_list = []

for T_eV, Rc, Zbar in zip(T_eV_list, Rc_list, Zbar_list):
    plasma = plasma_f(T_eV, Rc/ri_AU, Zbar )
    # plasma = plasma_f(T_eV, Rc/ri_AU, None )# TF
    plasma_list.append(plasma)

In [ ]:
guess_c_s_k_matrix = None
for Rc, plasma in zip(Rc_list[-1::], plasma_list[-1::]): 
    # if T_eV == 2.0:
    plasma.run_hnc(c_s_k_guess = guess_c_s_k_matrix)
    if plasma.hnc.tot_err_list[-1] > 1e-7:
        best_shift(plasma)
        solve_least_squares(plasma, np.linspace(plasma.hnc.k_array[0], plasma.hnc.k_array[-1], num=200))
        solve_least_squares(plasma, np.linspace(plasma.hnc.k_array[0], plasma.hnc.k_array[-1], num=400))
        save_plasma(plasma, Rc)
    else:
        save_plasma(plasma, Rc)
    # guess_c_s_k_matrix = plasma.hnc.c_s_k_matrix.copy()

### Plot and Comparison to QMD

In [ ]:

# for plasma, Rc in zip(plasma_list[4:5], Rc_list[4:5]):    
#     save_plasma(plasma, Rc)

## Paper Plots

In [ ]:
H1_GP = read_csv("GP_data/GP_H1_profile.dat", delim_whitespace=True, header=0)
H1_GP

In [ ]:
H1_CHNC = read_csv("results/CHNC_H_rs1.391_TeV2.000.dat", delim_whitespace=True, header=3)
H1_ri = 1.391
H1_Zbar = 1
H1_ne0 = H1_Zbar/(4/3*np.pi*H1_ri**3)
H1_CHNC

In [ ]:
def g_ei_plot(ax):
    ax.plot(H1_GP['x'], (H1_GP['ne_s']/H1_ne0-1)*4*π*(H1_GP['r'])**2,'k--', label="AA-NRL Free"  )
    ax.plot(H1_CHNC['r/ri'],(H1_CHNC['g_ei']-1)*4*π*(H1_CHNC['r/ri']*H1_ri)**2, linestyle=':', color=colors[1], label="PM-HNC Free"  )

    ax.set_xlim(0,1)
    ax.set_ylim(-5,5)
    ax.set_ylabel(r"$4\pi r^2 h_{ei}$")
    ax.set_xlabel(r"$r/a_i$")
    ax.legend()

fig, ax  = plt.subplots()
g_ei_plot(ax)

In [ ]:
def βu_ii_eff_plot(ax):

    ax.plot(H1_CHNC['r/ri'],H1_CHNC['βu_ii_eff'], label="PM-HNC"  )
    ax.plot(H1_CHNC['r/ri'],H1_CHNC['βu_OCP'],linestyle=(0,(5,1,1,1,1,1)),color=colors[3], label="Coulomb"  )
    ax.plot(H1_CHNC['r/ri'],H1_CHNC['βu_Yuk'],'-.', label="Yukawa"  ,color=colors[4])


    ax.set_xlim(0,4)
    
    ax.set_yscale('symlog', linthresh=1)
    ax.set_ylabel(r"$\beta u_{ii}$")
    ax.set_xlabel(r"$r/a_i$")
    ax.legend()
    
    
    custom_minor_ticks = list(np.linspace(-1,1,num=21)) + list(np.concatenate([list(np.linspace(1*10**n,10*10**n,num=10)) for n in range(2)]))
    ax.set_yticks(custom_minor_ticks, minor=True)
    ax.set_ylim(-0.1,1e3)
    
fig, ax  = plt.subplots()
βu_ii_eff_plot(ax)

In [ ]:
plasma = plasma_list[4]
def βu_ii_plot(ax):
    ax.plot(H1_CHNC['r/ri'], plasma.qsp.Ti*plasma.hnc.βu_r_matrix[0,0], label=r"$u_{ii}$"  )
    ax.plot(H1_CHNC['r/ri'],-plasma.qsp.Tie_c*plasma.hnc.βu_r_matrix[0,1],':', label=r"$-u_{ie}$"  )
    ax.plot(H1_CHNC['r/ri'], plasma.qsp.Te_c*plasma.hnc.βu_r_matrix[1,1],'--', label=r"$u_{ee}$"  )

    ax.set_xlim(0,1)
    ax.set_ylim(3e-1,1e2)
    ax.set_yscale('log')
    ax.set_ylabel(r"$u_{ij}/(Q_i Q_j)$ [au]")
    ax.set_xlabel(r"$r/a_i$")
    ax.legend()
    
fig, ax  = plt.subplots()
βu_ii_plot(ax)

In [ ]:
from scipy.interpolate import CubicSpline, PchipInterpolator

def g_ii_plot(ax):
    ax.plot(H1_CHNC['r/ri'],H1_CHNC['g_ii'], label="PM-HNC"  )
    ax.plot(H1_CHNC['r/ri'],H1_CHNC['OCP:g_ii'],':', label="OCP"  )
    ax.plot(H1_CHNC['r/ri'],H1_CHNC['Yuk:g_ii'],'--', label="Yukawa"  )

    # Data from community TCCW
    upper_data = read_csv("Fig5a_Upper.txt", delim_whitespace=True, header=1)
    lower_data = read_csv("Fig5a_Lower.txt", delim_whitespace=True, header=1)
    
    ax.fill_between(lower_data['r/ri'],lower_data['gii'], PchipInterpolator(upper_data['r/ri'], upper_data['gii'])(lower_data['r/ri']), alpha=0.5, color='k', label='Workshop DFT Spread')
    
    ax.set_xlim(0,4)
    ax.set_ylim(0,1.2)
    # ax.set_yscale('symlog', linthresh=0.1)
    ax.set_ylabel(r"$g_{ii}$")
    ax.set_xlabel(r"$r/a_i$")
    ax.legend()

# fig, ax  = plt.subplots( figsize=(8, 4.9443))
# g_ii_plot(ax)

def g_ij_plot(ax):
    ax.plot(H1_CHNC['r/ri'],H1_CHNC['g_ii'], label="PM-HNC ii"  )
    ax.plot(H1_CHNC['r/ri'],H1_CHNC['g_ei'],':', label="PM-HNC ei"  )
    ax.plot(H1_CHNC['r/ri'],H1_CHNC['g_ee'],'--', label="PM-HNC ee"  )
    
    ax.plot(H1_CHNC['r/ri'],H1_CHNC['OCP:g_ii'],linestyle=(0,(5,1,1,1,1,1)), label="Coulomb HNC ii" )
    ax.plot(H1_CHNC['r/ri'],H1_CHNC['Yuk:g_ii'],'-.', label="Yukawa HNC ii")
    
    ax.plot(H1_GP['x'], (H1_GP['ne_s']/H1_ne0), 'k--', label="AA-NRL ei "  )

    # Data from community TCCW
    upper_data = read_csv("Fig5a_Upper.txt", delim_whitespace=True, header=1)
    lower_data = read_csv("Fig5a_Lower.txt", delim_whitespace=True, header=1)
    
    ax.fill_between(lower_data['r/ri'],lower_data['gii'], PchipInterpolator(upper_data['r/ri'], upper_data['gii'])(lower_data['r/ri']), alpha=0.5, color='k', label='Workshop DFT Spread')
    ax.set_xlim(0,4)
    ax.set_ylim(0,1.5)
    # ax.set_yscale('symlog', linthresh=0.1)
    ax.set_ylabel(r"$g_{ij}$")
    ax.set_xlabel(r"$r/a_i$")
    ax.legend(loc='lower right',fontsize=12)

fig, ax  = plt.subplots( figsize=(8, 4.9443))
g_ij_plot(ax)

In [ ]:
fig, axs = plt.subplots(2,2, figsize=(12, 9))
βu_ii_plot(axs[0,0])
βu_ii_eff_plot(axs[0,1])

g_ei_plot(axs[1,0])
g_ij_plot(axs[1,1])
axs[0,1].axhline(y=1,color='k',linewidth=0.4)
axs[0,1].fill_between(np.linspace(0,10),-1,1,color='k',alpha=0.05)
axs[0,1].text(0.03, 1.10, "log-scale",fontsize=14)
axs[0,1].text(0.03, 0.8, "linear-scale",fontsize=14)
# Subfigure Labels
axs[0,0].text(0.05, 0.95, "a)", transform=axs[0,0].transAxes, fontsize=18)
axs[0,1].text(0.05, 0.95, "b)", transform=axs[0,1].transAxes, fontsize=18)
axs[1,0].text(0.05, 0.95, "c)", transform=axs[1,0].transAxes, fontsize=18)
axs[1,1].text(0.05, 0.95, "d)", transform=axs[1,1].transAxes, fontsize=18)

plt.tight_layout()
plt.savefig("H1_gβu_plot.png", dpi=400)


## Specific Heat

In [ ]:
plasma_list[-1].run_hnc()

In [ ]:
plasma_list[-1].get_effective_ion_cp()

In [ ]:
plasma_list[-1].get_effective_ion_cp(ε_derivative=1e-6, ε_table=1e-16, N_table=2)

In [ ]:
plasma_list[-1].ni_cc, plasma_list[-1].ni_cc 

In [ ]:
plasma_list[-1].n_table, plasma_list[-1].T_table 

In [ ]:
plasma_list[-1].run_hnc(c_s_k_guess = plasma_list[-1].hnc.c_s_k_matrix.copy())

In [ ]:
plasma_list[-1].get_effective_ion_cp(ε_derivative=1e-8, ε_table=1e-6, N_table=2)

In [ ]:
plasma_list[-1].get_effective_ion_cp(ε_derivative=1e-6, ε_table=1e-4, N_table=3)

## Conductivity from Ziman

https://arxiv.org/pdf/1508.06101.pdf eq.5 in Hansen & McDonald the eqn's 10.2.4, 10.2.11

$$
\sigma^{-1} = \rho= \frac{1}{32 \sqrt{\pi} Z} \frac{m_e^2}{(2 m_e T_e^c)^{3/2}} \int^{\infty}_0 e^{-q^2/(8 m_e T_e^c)}  |\tilde{V}(q)|^2 S_{ii}(q) q^3 dq
$$
where the screened potential 
$$\tilde{V}(q) = V(q)/\epsilon,$$
for $$\frac{1}{\epsilon}  = 1 + \frac{4 \pi}{k^2}\chi_{ee}(k) $$ 
and 
$$\chi_{ee}(k) = -\beta_e^c n_e S_{ee}(k) $$

In [ ]:
from scipy.integrate import simpson

def conductivity(plasma):
    q_array = plasma.hnc.k_array/plasma.qsp.ri 
    
    rhosum = np.sum(plasma.hnc.rho/plasma.qsp.ri**3)
    x = plasma.hnc.rho/plasma.qsp.ri**3/rhosum
    S_matrix = np.diag(x)[:,:,np.newaxis] + rhosum * x[:,np.newaxis,np.newaxis]*x[np.newaxis,:,np.newaxis]*plasma.hnc.h_k_matrix
    χ_ee = -plasma.qsp.βe_c*plasma.qsp.ne*S_matrix[1,1]
    kTF = plasma.qsp.get_κ()/plasma.qsp.ri
    # χ_ee = -q_array**2/(4*π)* kTF**2/(kTF**2 + q_array**2)
    ε = 1/( 1 + 4*π/q_array**2 * χ_ee)
    Vtilde = -plasma.Zbar/(plasma.hnc.r_array*plasma.qsp.ri)/ε
    # print(ε)
    integrand = np.exp(-plasma.qsp.βe_c * q_array**2/8) * np.abs(Vtilde)**2 * S_matrix[0,0] * q_array**3
    ρ_AU = 1/(  32*np.sqrt(π)*plasma.Zbar*(2*plasma.qsp.Te_c)**1.5) * simpson(integrand, q_array)  
    σ_AU = 1/ρ_AU
    σ_Spercm = σ_AU * AU_to_Siemens/AU_to_cm
    return σ_Spercm
print(f"Conductivity: σ = {conductivity(plasma):0.3e}")

In [ ]:
fig, ax = plt.subplots( facecolor='white')

ax.plot(T_eV_list, [conductivity(plasma) for plasma in plasma_list], label='Ziman')

ax.set_ylabel(r"$\sigma $[S/cm]")
ax.set_xlabel(r"$T$ [eV]")


# Inset Text Box
# props = dict(boxstyle='round', facecolor='white', alpha=0.5)
# inset_text = f"H\n {sci_notation(ni_cc,3)} [cc$^{{-1}}$]" 
# ax.text(0.35, 0.95, inset_text, transform=ax.transAxes, fontsize=14,
#         verticalalignment='top', bbox=props)

# ax.errorbar([tccw_TeV],[tccw_η_cgs], yerr=[tccw_η_cgs_err], barsabove=True, capsize=5, marker='.', label='MD Simulation')

# Scales
ax.set_yscale('log')
ax.set_xscale('log')
# ax.set_ylim(5e4,5e6)

# Legend
ax.legend()
